In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def load_evaluations(base_dir="evals"):
    evaluations = {}
    for model in os.listdir(base_dir):
        model_path = os.path.join(base_dir, model)
        if os.path.isdir(model_path):
            evaluations[model] = {}
            for file in os.listdir(model_path):
                if file.endswith('_evaluation.npy'):
                    xai_method = file.split('_')[0]
                    file_path = os.path.join(model_path, file)
                    evaluations[model][xai_method] = np.load(file_path)
    return evaluations

# Load all evaluations
all_evaluations = load_evaluations()

# Compute averages and standard deviations
def compute_stats(eval_array):
    return np.mean(eval_array, axis=0), np.std(eval_array, axis=0)

stats = {model: {xai: compute_stats(eval_array)
                 for xai, eval_array in model_evals.items()}
         for model, model_evals in all_evaluations.items()}

In [2]:
np.set_printoptions(suppress=True, precision=4)

# Sanity / correctness check

In [5]:
def has_nan(arr):
    return np.isnan(arr).any()

for model in all_evaluations:
    for xai_method in all_evaluations[model]:
        # Verify no score is NaN
        if has_nan(all_evaluations[model][xai_method]):
            print(f"Model {model} with XAI method {xai_method} has NaNs")
        # check if all values are in the range [-1, 1]
        if (all_evaluations[model][xai_method] > 1).any() or (all_evaluations[model][xai_method] < -1).any():
            print(f"Model {model} with XAI method {xai_method} has values outside of [-1, 1]")

# Evaluation

In [7]:
all_evaluations["llama3"]["attnlrp"][36, 3, 1]

0.12377475202083588

In [6]:
all_evaluations["unllama3"]["attnlrp"][71][:, 0]

array([ 0.0003,  0.7505, -0.0005, -0.1261, -0.0006, -0.0007,  0.    ,
        0.0001, -0.0003, -0.0003, -0.0923, -0.2396, -0.    , -0.006 ,
       -0.0123, -0.0039, -0.0061])